# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
def info_geo_data(geo_data):
    print('Первые 5 строк датасета')
    display(geo_data.head())
    print()
    print('Информация о датасете')
    display(geo_data.info())
    print()
    print('Описательная статистика датасета')
    display(geo_data.describe())
    print()
    print('Количество дубликатов:', geo_data.duplicated().sum())
    print()
    print('Коэффициент корреляции')
    display(geo_data.corr())
    print('-------------------------------------------------------')

In [4]:
info_geo_data(geo_data_0)
info_geo_data(geo_data_1)
info_geo_data(geo_data_2)

Первые 5 строк датасета


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Информация о датасете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None


Описательная статистика датасета


...


Описательная статистика датасета


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



Количество дубликатов: 0

Коэффициент корреляции


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


-------------------------------------------------------


<b>Вывод</b>  

Вывел основную информацию о данных.
Пропусков - нет, дубликатов - нет. Корреляция 2 региона показывает, что количество нефти сильно зависит от признака f2. Может быть прибыльным регионом.

## Обучение и проверка модели

In [5]:
def split_train (geo_data):
    features = geo_data.drop(['product','id'], axis=1)
    target = geo_data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345
    )
    
    line_reg = LinearRegression()
    line_reg.fit(features_train, target_train)
    predicted_valid = line_reg.predict(features_valid)
    mean_product_predicted_valid = sum(predicted_valid) / len(predicted_valid)
    rmse = mean_squared_error(predicted_valid, target_valid) ** 0.5
    
    print('Средний запас предсказанного сырья:', round(mean_product_predicted_valid, 3))
    print('RMSE:', round(rmse, 3))
    
    return predicted_valid, target_valid, rmse

In [6]:
print('Регион 1')
predicted_valid_0, target_valid_0, rmse_0 = split_train(geo_data_0)
print()
print('Регион 2')
predicted_valid_1, target_valid_1, rmse_1 = split_train(geo_data_1)
print()
print('Регион 3')
predicted_valid_2, target_valid_2, rmse_2 = split_train(geo_data_2)

Регион 1
Средний запас предсказанного сырья: 92.593
RMSE: 37.579

Регион 2
Средний запас предсказанного сырья: 68.729
RMSE: 0.893

Регион 3
Средний запас предсказанного сырья: 94.965
RMSE: 40.03


<b>Вывод</b>  

Данные разделены на выборки в отношении 75:25.   
Обучена модель линейной регрессии на валидационной выборке.  
Найден средний запас сырья в регионах. Самый стабильный регион - второй. Среднеквадратическая ошибка минимальна.

## Подготовка к расчёту прибыли

In [7]:
budget = 10000000000
income_per_unit = 450000
n = 200

quantity_barrels = budget / (income_per_unit * n)
print('Количество баррелей, которое нужно добыть с одной скважины, чтобы отбить затраты на разработку:', 
      round(quantity_barrels, 2)
     )

Количество баррелей, которое нужно добыть с одной скважины, чтобы отбить затраты на разработку: 111.11


In [8]:
def comparison(data):
    mean_product = data.mean()
    print('Среднее количеcтво нефти в регионе', round(mean_product, 2))
    if mean_product > quantity_barrels:
        print('Хватит нефти')
        print('------------------------------------------------------------')
    else:
        print('Нужно больше нефти!')
        print('------------------------------------------------------------')

print('Регион 1')
comparison(geo_data_0['product'])
print()
print('Регион 2')
comparison(geo_data_0['product'])
print()
print('Регион 3')
comparison(geo_data_0['product'])

Регион 1
Среднее количеcтво нефти в регионе 92.5
Нужно больше нефти!
------------------------------------------------------------

Регион 2
Среднее количеcтво нефти в регионе 92.5
Нужно больше нефти!
------------------------------------------------------------

Регион 3
Среднее количеcтво нефти в регионе 92.5
Нужно больше нефти!
------------------------------------------------------------


<b>Вывод</b>  

Найдено среднее количество запасов нефти по регионам и количество баррелей, которое нужно добыть, чтобы отбить затраты на разработку скважин.

## Расчёт прибыли и рисков 

In [16]:
target_valid_0 = pd.Series(target_valid_0).reset_index(drop=True)
predicted_valid_0 = pd.Series(predicted_valid_0).reset_index(drop=True)
target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
predicted_valid_1 = pd.Series(predicted_valid_1).reset_index(drop=True)
target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
predicted_valid_2 = pd.Series(predicted_valid_2).reset_index(drop=True)

def revenue (target, predictions):
    predicted_valid = predictions.sort_values(ascending=False)[:200]
    target_valid = target[predicted_valid.index]
    return income_per_unit * target_valid.sum() - budget 

In [10]:
revenue(target_valid_0, predicted_valid_0)

3320826043.1398506

In [11]:
revenue(target_valid_1, predicted_valid_1)

2415086696.681511

In [12]:
revenue(target_valid_2, predicted_valid_2)

2710349963.5998325

In [17]:
target_valid = [target_valid_0, target_valid_1, target_valid_2]
predicted_valid = [predicted_valid_0, predicted_valid_1, predicted_valid_2]
regions = ['Первый регион', 'Второй регион', 'Третий регион']

state = np.random.RandomState(12345)

for m in range(len(target_valid)):
    values = []
    for i in tqdm(range(1000), desc='Progress'):
        target_subsample = target_valid[m].sample(500, replace=True, random_state=state)
        predictions_subsample = predicted_valid[m].loc[target_subsample.index]
        values.append(revenue(target_subsample.reset_index(drop=True), predictions_subsample.reset_index(drop=True)))

    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)

    mean = values.mean()
    
    loss_probability = sum(values < 0) / values.count() * 100
    
    print(regions[m])
    print("Средняя возможная прибыль:", mean)
    print("95%-доверительный интервал:", "от", lower, "до", upper)
    print("Вероятность убытков:", loss_probability, "%")
    print()

Progress:   0%|          | 0/1000 [00:00<?, ?it/s]

Первый регион
Средняя возможная прибыль: 396164984.8023711
95%-доверительный интервал: от -111215545.89049526 до 909766941.5534226
Вероятность убытков: 6.9 %



Progress:   0%|          | 0/1000 [00:00<?, ?it/s]

Второй регион
Средняя возможная прибыль: 461155817.2772397
95%-доверительный интервал: от 78050810.7517417 до 862952060.2637234
Вероятность убытков: 0.7000000000000001 %



Progress:   0%|          | 0/1000 [00:00<?, ?it/s]

Третий регион
Средняя возможная прибыль: 392950475.17060447
95%-доверительный интервал: от -112227625.37857565 до 934562914.5511636
Вероятность убытков: 6.5 %



<b>Вывод</b>  

Написана функция для рассчета прибыли. Рассчитаны: средняя возможная прибыль, 95% доверительный интервал и вероятность убытков. 
Самый выгодный регион - второй.

## Общий вывод  

В данной работе были исследованы три региона где добывается нефть, чтобы построить модель для определения наиболее выгодного
региона для добычи нефти.  
Данные были проверены на наличие пропусков и дубликатов. Корреляция показала, что во втором регионе количество запасов напрямую зависит от признака f2.  
Данные были разделены на выборки и использованы для обучения модели линейной регрессии и получения ее предсказаний. По среднеквадратичной ошибке наиболее стабильный регион - второй.  
Посчитано минимальное количество добываемых баррелей для отбития затрат. Также вычислен средний запас продукта в каждом регионе.   
Рассчитаны прибыли и вероятности убытков для всех регионов.  Самый прибыльный и стабильный регион - второй.